In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
# import cudf
# import cuml
# from cuml.feature_extraction.text import TfidfVectorizer as cuTfidfVectorizer
# from cuml.linear_model import LogisticRegression as cuLogisticRegression
# from cuml.linear_model import MBSGDClassifier as cuSGDClassifier
# from cuml.naive_bayes import MultinomialNB as cuMultinomialNB
# import xgboost as xgb
# from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report



In [2]:
train = pd.read_csv('/kaggle/input/generated-text/data_split/train.csv')
test = pd.read_csv('/kaggle/input/generated-text/data_split/test.csv')
val = pd.read_csv('/kaggle/input/generated-text/data_split/val.csv')

import string
import re
import nltk
from nltk.corpus import stopwords
Stopwords = set(stopwords.words('english'))
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    text = re.sub('\s+',' ',text)
    
    new_text =[]
    for word in text.split():
        if word not in Stopwords:
            new_text.append(word)
    
    ps = nltk.stem.porter.PorterStemmer()
    new_text = [ps.stem(word) for word in new_text]
    #lem = nltk.stem.wordnet.WordNetLemmatizer()
    #new_text = [lem.lemmatize(word) for word in new_text]
    
    text =" ".join(new_text)
    return(text)

train['clean_text'] = train['text'].apply(lambda x: preprocess_text(x))	
val['clean_text'] = val['text'].apply(lambda x: preprocess_text(x))	
test['clean_text'] = test['text'].apply(lambda x: preprocess_text(x))


In [3]:
# x = dfe[['clean_text']]
# y = dfe[['generated']]
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=dfe['generated'])
x_train = train[['clean_text']]
x_val = val[['clean_text']]
x_test =  test[['clean_text']]
y_train = train[['generated']]
y_val = val[['generated']]
y_test = test[['generated']]
df = pd.concat([x_train['clean_text'],x_val['clean_text'], x_test['clean_text'],test['clean_text']] ,axis=0)

In [4]:
#vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=50000, min_df=0.1, max_df=0.9, sublinear_tf=True)
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=50000, sublinear_tf=True)
X = vectorizer.fit_transform(df)

# vectorizer = cuTfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True)
# X = vectorizer.fit_transform(df)

In [5]:
# lr_model = LogisticRegression(solver="liblinear")
# sgd_model = SGDClassifier(max_iter=1000, tol=1e-3, loss="modified_huber")
# nb_model = MultinomialNB()
# xgb_model = XGBClassifier()

# # Create the ensemble model
# ensemble = VotingClassifier(estimators=[('lr', lr_model),('sgd', sgd_model),('nb',nb_model),('xgb',xgb_model)], voting='soft')

# # lr_model = cuLogisticRegression()
# # sgd_model = cuSGDClassifier()
# # nb_model = cuMultinomialNB()
# # xgb_model = xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0)

# # # Create the ensemble model
# # ensemble = VotingClassifier(estimators=[
# #     ('lr', lr_model),
# #     ('sgd', sgd_model),
# #     ('nb', nb_model),
# #     ('xgb', xgb_model)
# # ], voting='soft')

In [6]:
# ensemble.fit(X[:x_train.shape[0]], y_train.generated)

In [7]:
# # Predictions on the validation set
# y_val_pred = ensemble.predict(X[x_train.shape[0]:x_train.shape[0] + x_val.shape[0]])

# # Predictions on the test set
# y_test_pred = ensemble.predict(X[x_train.shape[0] + x_val.shape[0]:x_train.shape[0] + x_val.shape[0]+ x_test.shape[0]])

In [8]:
# y_val_pred = ensemble.estimators_[3].predict(X[x_train.shape[0]:x_train.shape[0] + x_val.shape[0]])
# y_test_pred = ensemble.estimators_[3].predict(X[x_train.shape[0] + x_val.shape[0]:x_train.shape[0] + x_val.shape[0]+ x_test.shape[0]])

In [9]:
lr_model = LogisticRegression(solver="liblinear")
lr_model.fit(X[:x_train.shape[0]], y_train.generated)
y_val_pred_lr = lr_model.predict(X[x_train.shape[0]:x_train.shape[0] + x_val.shape[0]])
y_test_pred_lr = lr_model.predict(X[x_train.shape[0] + x_val.shape[0]:x_train.shape[0] + x_val.shape[0]+ x_test.shape[0]])

val_accuracy_lr = accuracy_score(y_val['generated'], y_val_pred_lr)
val_precision_lr = precision_score(y_val['generated'], y_val_pred_lr)
val_recall_lr = recall_score(y_val['generated'], y_val_pred_lr)
val_f1_lr = f1_score(y_val['generated'], y_val_pred_lr)

test_accuracy_lr = accuracy_score(y_test['generated'], y_test_pred_lr)
test_precision_lr = precision_score(y_test['generated'], y_test_pred_lr)
test_recall_lr = recall_score(y_test['generated'], y_test_pred_lr)
test_f1_lr = f1_score(y_test['generated'], y_test_pred_lr)

print(f'Validation Accuracy: {val_accuracy_lr}')
print(f'Validation Precision: {val_precision_lr}')
print(f'Validation Recall: {val_recall_lr}')
print(f'Validation F1-Score: {val_f1_lr}')


print(f'Test Accuracy: {test_accuracy_lr}')
print(f'Test Precision: {test_precision_lr}')
print(f'Test Recall: {test_recall_lr}')
print(f'Test F1-Score: {test_f1_lr}')
print("Test Confusion Matrix:")
print(confusion_matrix(y_test['generated'], y_test_pred_lr))


Validation Accuracy: 0.8652580971659919
Validation Precision: 0.8778195488721805
Validation Recall: 0.9585385878489326
Validation F1-Score: 0.9164050235478806
Test Accuracy: 0.8700189753320683
Test Precision: 0.8781182111019552
Test Recall: 0.9647736625514404
Test F1-Score: 0.9194086042589905
Test Confusion Matrix:
[[ 2033  1627]
 [  428 11722]]


In [10]:
sgd_model = SGDClassifier(max_iter=1000, tol=1e-3, loss="modified_huber")
sgd_model.fit(X[:x_train.shape[0]], y_train.generated)
y_val_pred_sgd = sgd_model.predict(X[x_train.shape[0]:x_train.shape[0] + x_val.shape[0]])
y_test_pred_sgd = sgd_model.predict(X[x_train.shape[0] + x_val.shape[0]:x_train.shape[0] + x_val.shape[0]+ x_test.shape[0]])

val_accuracy_sgd = accuracy_score(y_val['generated'], y_val_pred_sgd)
val_precision_sgd = precision_score(y_val['generated'], y_val_pred_sgd)
val_recall_sgd = recall_score(y_val['generated'], y_val_pred_sgd)
val_f1_sgd = f1_score(y_val['generated'], y_val_pred_sgd)

test_accuracy_sgd = accuracy_score(y_test['generated'], y_test_pred_sgd)
test_precision_sgd = precision_score(y_test['generated'], y_test_pred_sgd)
test_recall_sgd = recall_score(y_test['generated'], y_test_pred_sgd)
test_f1_sgd = f1_score(y_test['generated'], y_test_pred_sgd)

print(f'Validation Accuracy: {val_accuracy_sgd}')
print(f'Validation Precision: {val_precision_sgd}')
print(f'Validation Recall: {val_recall_sgd}')
print(f'Validation F1-Score: {val_f1_sgd}')

print(f'Test Accuracy: {test_accuracy_sgd}')
print(f'Test Precision: {test_precision_sgd}')
print(f'Test Recall: {test_recall_sgd}')
print(f'Test F1-Score: {test_f1_sgd}')
print("Test Confusion Matrix:")
print(confusion_matrix(y_test['generated'], y_test_pred_sgd))


Validation Accuracy: 0.8619053643724697
Validation Precision: 0.8690291620524179
Validation Recall: 0.9664203612479475
Validation F1-Score: 0.9151409135082605
Test Accuracy: 0.8656546489563567
Test Precision: 0.8698539176626826
Test Recall: 0.9703703703703703
Test F1-Score: 0.9173669467787113
Test Confusion Matrix:
[[ 1896  1764]
 [  360 11790]]


In [11]:
nb_model = MultinomialNB()
nb_model.fit(X[:x_train.shape[0]], y_train.generated)
y_val_pred_nb = nb_model.predict(X[x_train.shape[0]:x_train.shape[0] + x_val.shape[0]])
y_test_pred_nb = nb_model.predict(X[x_train.shape[0] + x_val.shape[0]:x_train.shape[0] + x_val.shape[0]+ x_test.shape[0]])

val_accuracy_nb = accuracy_score(y_val['generated'], y_val_pred_nb)
val_precision_nb = precision_score(y_val['generated'], y_val_pred_nb)
val_recall_nb = recall_score(y_val['generated'], y_val_pred_nb)
val_f1_nb = f1_score(y_val['generated'], y_val_pred_nb)

test_accuracy_nb = accuracy_score(y_test['generated'], y_test_pred_nb)
test_precision_nb = precision_score(y_test['generated'], y_test_pred_nb)
test_recall_nb = recall_score(y_test['generated'], y_test_pred_nb)
test_f1_nb = f1_score(y_test['generated'], y_test_pred_nb)

print(f'Validation Accuracy: {val_accuracy_nb}')
print(f'Validation Precision: {val_precision_nb}')
print(f'Validation Recall: {val_recall_nb}')
print(f'Validation F1-Score: {val_f1_nb}')

print(f'Test Accuracy: {test_accuracy_nb}')
print(f'Test Precision: {test_precision_nb}')
print(f'Test Recall: {test_recall_nb}')
print(f'Test F1-Score: {test_f1_nb}')
print("Test Confusion Matrix:")
print(confusion_matrix(y_test['generated'], y_test_pred_nb))

Validation Accuracy: 0.7782135627530364
Validation Precision: 0.8672311600338696
Validation Recall: 0.8408866995073891
Validation F1-Score: 0.8538557732388495
Test Accuracy: 0.7797596457938014
Test Precision: 0.869669054930058
Test Recall: 0.8391769547325103
Test F1-Score: 0.8541509592024796
Test Confusion Matrix:
[[ 2132  1528]
 [ 1954 10196]]


In [12]:
xgb_model = XGBClassifier()
xgb_model.fit(X[:x_train.shape[0]], y_train.generated)
y_val_pred_xgb = xgb_model.predict(X[x_train.shape[0]:x_train.shape[0] + x_val.shape[0]])
y_test_pred_xgb = xgb_model.predict(X[x_train.shape[0] + x_val.shape[0]:x_train.shape[0] + x_val.shape[0]+ x_test.shape[0]])

val_accuracy_xgb = accuracy_score(y_val['generated'], y_val_pred_xgb)
val_precision_xgb = precision_score(y_val['generated'], y_val_pred_xgb)
val_recall_xgb = recall_score(y_val['generated'], y_val_pred_xgb)
val_f1_xgb = f1_score(y_val['generated'], y_val_pred_xgb)

test_accuracy_xgb = accuracy_score(y_test['generated'], y_test_pred_xgb)
test_precision_xgb = precision_score(y_test['generated'], y_test_pred_xgb)
test_recall_xgb = recall_score(y_test['generated'], y_test_pred_xgb)
test_f1_xgb = f1_score(y_test['generated'], y_test_pred_xgb)

print(f'Validation Accuracy: {val_accuracy_xgb}')
print(f'Validation Precision: {val_precision_xgb}')
print(f'Validation Recall: {val_recall_xgb}')
print(f'Validation F1-Score: {val_f1_xgb}')

print(f'Test Accuracy: {test_accuracy_xgb}')
print(f'Test Precision: {test_precision_xgb}')
print(f'Test Recall: {test_recall_xgb}')
print(f'Test F1-Score: {test_f1_xgb}')
print("Test Confusion Matrix:")
print(confusion_matrix(y_test['generated'], y_test_pred_xgb))

Validation Accuracy: 0.8421052631578947
Validation Precision: 0.8495524112041583
Validation Recall: 0.9661740558292282
Validation F1-Score: 0.9041180086047942
Test Accuracy: 0.8433902593295383
Test Precision: 0.8488892094633583
Test Recall: 0.9686419753086419
Test F1-Score: 0.9048204812793111
Test Confusion Matrix:
[[ 1565  2095]
 [  381 11769]]


In [13]:
# # Calculate validation accuracy
# accuracy = accuracy_score(val_labels, val_preds)
# conf_matrix = confusion_matrix(val_labels, val_preds)
# classification_rep = classification_report(val_labels, val_preds)
# print(f'Accuracy: {accuracy}')
# print(f'Confusion Matrix:\n{conf_matrix}')
# print(f'Classification Report:\n{classification_rep}')

In [14]:
# # Accuracy
# val_accuracy = accuracy_score(Y_val['generated'], y_val_pred)
# print(f'Validation Accuracy: {val_accuracy}')

# # Precision, Recall, F1-Score
# val_precision = precision_score(Y_val['generated'], y_val_pred, average='weighted')
# val_recall = recall_score(Y_val['generated'], y_val_pred, average='weighted')
# val_f1 = f1_score(Y_val['generated'], y_val_pred, average='weighted')

# print(f'Validation Precision: {val_precision}')
# print(f'Validation Recall: {val_recall}')
# print(f'Validation F1-Score: {val_f1}')

# # Confusion Matrix and Classification Report
# print("Validation Confusion Matrix:")
# print(confusion_matrix(Y_val['generated'], y_val_pred))

# print("\nValidation Classification Report:")
# print(classification_report(Y_val['generated'], y_val_pred))


In [15]:
# # Accuracy
# test_accuracy = accuracy_score(y_test['generated'], y_test_pred)
# print(f'Test Accuracy: {test_accuracy}')

# # Precision, Recall, F1-Score
# test_precision = precision_score(y_test['generated'], y_test_pred, average='weighted')
# test_recall = recall_score(y_test['generated'], y_test_pred, average='weighted')
# test_f1 = f1_score(y_test['generated'], y_test_pred, average='weighted')

# print(f'Test Precision: {test_precision}')
# print(f'Test Recall: {test_recall}')
# print(f'Test F1-Score: {test_f1}')

# # Confusion Matrix and Classification Report
# print("Test Confusion Matrix:")
# print(confusion_matrix(y_test['generated'], y_test_pred))

# print("\nTest Classification Report:")
# print(classification_report(y_test['generated'], y_test_pred))